<a href="https://colab.research.google.com/github/Oba-max22/learner/blob/master/StageCquiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
import pandas as pd

In [70]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00471/Data_for_UCI_named.csv')
df.describe(include='all')

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unstable
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6380
mean,5.250000,5.250001,5.250004,5.249997,3.750000,-1.250000,-1.250000,-1.250000,0.525000,0.525000,0.525000,0.525000,0.015731,NaN
std,2.742548,2.742549,2.742549,2.742556,0.752160,0.433035,0.433035,0.433035,0.274256,0.274255,0.274255,0.274255,0.036919,NaN
min,0.500793,0.500141,0.500788,0.500473,1.582590,-1.999891,-1.999945,-1.999926,0.050009,0.050053,0.050054,0.050028,-0.080760,NaN
25%,2.874892,2.875140,2.875522,2.874950,3.218300,-1.624901,-1.625025,-1.624960,0.287521,0.287552,0.287514,0.287494,-0.015557,NaN
50%,5.250004,5.249981,5.249979,5.249734,3.751025,-1.249966,-1.249974,-1.250007,0.525009,0.525003,0.525015,0.525002,0.017142,NaN
75%,7.624690,7.624893,7.624948,7.624838,4.282420,-0.874977,-0.875043,-0.875065,0.762435,0.762490,0.762440,0.762433,0.044878,NaN


In [71]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [72]:
df = df.drop(columns=['stab'])

In [73]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,unstable


In [74]:
df['stabf'].value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [75]:
df.isna().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stabf    0
dtype: int64

In [76]:
df.stabf.value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [77]:
X = df.drop(columns='stabf')
y = df['stabf']

In [78]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
y_train.value_counts()

unstable    5092
stable      2908
Name: stabf, dtype: int64

In [79]:
y_test.value_counts()

unstable    1288
stable       712
Name: stabf, dtype: int64

In [80]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_sc = scaler.fit_transform(x_train)
x_train_scale = pd.DataFrame(x_train_sc, columns = x_train.columns)

In [81]:
x_test_sc = scaler.transform(x_test)
x_test_scale = pd.DataFrame(x_test_sc, columns = x_test.columns)

In [82]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state =1 )
rfc.fit(x_train_scale, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [83]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
new_predictions = rfc.predict(x_test_scale)
cnf_mat = confusion_matrix(y_true=y_test, y_pred=new_predictions, labels=['stable', 'unstable'])
cnf_mat

array([[ 625,   87],
       [  55, 1233]])

In [84]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rfc, x_train_scale, y_train, cv=5, scoring='f1_macro')
scores

array([0.91143756, 0.91136454, 0.91564855, 0.90214725, 0.91555674])

In [104]:
accuracy = accuracy_score(y_true=y_test, y_pred=new_predictions)
print (accuracy)
print('Accuracy: {}'.format(round(accuracy*100), 3))

0.929
Accuracy: 93.0


In [86]:
precision = precision_score(y_true=y_test, y_pred=new_predictions, pos_label='stable')
print('Precision: {}'.format(round(precision*100), 2))

Precision: 92.0


In [87]:
recall = recall_score(y_true=y_test, y_pred=new_predictions, pos_label='stable')
print('Recall: {}'.format(round(recall*100), 2))

Recall: 88.0


In [88]:
f1 = f1_score(y_true=y_test, y_pred=new_predictions, pos_label='stable')
print('F1: {}'.format(round(f1*100), 2))

F1: 90.0


In [89]:
from sklearn.ensemble import ExtraTreesClassifier
clf_model = ExtraTreesClassifier(n_estimators = [50, 100, 300, 500, 1000],
                           min_samples_split = [2, 3, 5, 7, 9], 
                           min_samples_leaf = [1, 2, 4, 6, 8], 
                           max_features = ['auto', 'sqrt', 'log2', None],
                           random_state=1)
print(clf_model)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None,
                     max_features=['auto', 'sqrt', 'log2', None],
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=[1, 2, 4, 6, 8],
                     min_samples_split=[2, 3, 5, 7, 9],
                     min_weight_fraction_leaf=0.0,
                     n_estimators=[50, 100, 300, 500, 1000], n_jobs=None,
                     oob_score=False, random_state=1, verbose=0,
                     warm_start=False)
